In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from utils.Dataset import Dataset
from net.networks import ResNet10
import random
import torch.random
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score,average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score
import joblib
torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)

EHRs_DrugRel_Lab = pd.read_csv("preprocessed_data(dummy)/(dummy)EHRs_DrugRel_Lab.csv")
Lab_col=EHRs_DrugRel_Lab.columns[101:136]
DrugRel_col=EHRs_DrugRel_Lab.columns[346:1741]
EHRs_DrugRel=EHRs_DrugRel_Lab.drop(Lab_col.values,axis=1)
EHRs=EHRs_DrugRel.drop(DrugRel_col,axis=1)

data={"EHRs":EHRs,"EHRs_DrugRel":EHRs_DrugRel,"EHRs_DrugRel_Lab":EHRs_DrugRel_Lab}

for d in data.keys():
    data_df=data[d]
    print(d,": ")
    data_df=data_df.drop(["Sepsis_Date"],axis=1)
    padding = pd.DataFrame(0*np.ones((len(data_df), 1764-len(data_df.columns)+1)))
    data_df = pd.concat([data_df,padding],axis=1) 
    test_data=data_df
    test_feature = test_data.drop(["Label"], axis=1)
    test_label = test_data[["Label"]]        
    scaler = joblib.load("trained_model/"+d+"_scaler_resnet10.pkl")
    test_feature = scaler.transform(test_feature)
    test_feature=test_feature.reshape(test_feature.shape[0],1,42,42)
      
    model=ResNet10()        
    model=model.cuda()
    model.load_state_dict(torch.load("trained_model/"+d+"_resnet10.pt"))

    y_pre=torch.round(model(torch.cuda.FloatTensor(test_feature))).cpu().detach().numpy()
    y_proba=model(torch.cuda.FloatTensor(test_feature)).cpu().detach().numpy()
    
    y_label = test_label.values
    
    test_acc = (y_pre==y_label).sum()/len(y_label)
    AUROC = roc_auc_score(y_label, y_proba)
    AUPRC = average_precision_score(y_label, y_proba)
    precision = precision_score(y_label, y_pre, pos_label=1)
    recall = recall_score(y_label, y_pre)
    f1_score_ = f1_score(y_label, y_pre)
    
    print("ResNet10 : ")
    print("  Acc      : ",np.round(test_acc ,3)) 
    print("  AUROC    : ",np.round(AUROC    ,3))
    print("  AUPRC    : ",np.round(AUPRC    ,3))
    print("  Precision: ",np.round(precision,3))
    print("  Recall   : ",np.round(recall   ,3))
    print("  F1-score : ",np.round(f1_score_,3))
    print("\n") 

EHRs : 
ResNet10 : 
  Acc      :  0.9
  AUROC    :  0.84
  AUPRC    :  0.81
  Precision:  0.833
  Recall   :  1.0
  F1-score :  0.909


EHRs_DrugRel : 
ResNet10 : 
  Acc      :  1.0
  AUROC    :  1.0
  AUPRC    :  1.0
  Precision:  1.0
  Recall   :  1.0
  F1-score :  1.0


EHRs_DrugRel_Lab : 
ResNet10 : 
  Acc      :  0.9
  AUROC    :  1.0
  AUPRC    :  1.0
  Precision:  0.833
  Recall   :  1.0
  F1-score :  0.909


